In [2]:
import pyodbc
import pandas as pd
from sqlalchemy import create_engine

In [3]:
reported_cases_data_df = pd.read_excel(r"E:\vaccine_data\reported-cases-data.xlsx")
coverage_data_df = pd.read_excel(r"E:\vaccine_data\coverage-data.xlsx")
incidence_rate_df = pd.read_excel(r"E:\vaccine_data\incidence-rate-data.xlsx")
vaccine_introduction_df = pd.read_excel(r"E:\vaccine_data\vaccine-introduction-data.xlsx")
vaccine_schedule_df = pd.read_excel(r"E:\vaccine_data\vaccine-schedule-data.xlsx")

In [5]:
# Connection details
server = r'INSPIRON-5518\MSQLSERVER'   # your server name
database = 'vaccination'                    # replace with your database name
username = 'sa'                        # your SQL username
password = '7809'             # your SQL password

# Create the connection
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    f'SERVER={server};'
    f'DATABASE={database};'
    'Trusted_Connection=yes;'
)

# Now create cursor from the connection
cursor = conn.cursor()
print("connected to the sql server")


# Create SQLAlchemy engine for bulk insert
engine = create_engine(
    f"mssql+pyodbc://{server}/{database}?driver=ODBC%20Driver%2017%20for%20SQL%20Server&trusted_connection=yes"
)

connected to the sql server


In [6]:
# Load data from SQL Server into DataFrames
print("Loading data from SQL Server into DataFrames...")

reported_cases_df = pd.read_sql("SELECT * FROM reported_cases_data", con=engine)
coverage_df = pd.read_sql("SELECT * FROM coverage_data", con=engine)
incidence_rate_df = pd.read_sql("SELECT * FROM incidence_rate_data", con=engine)
vaccine_introduction_df_loaded = pd.read_sql("SELECT * FROM vaccine_introduction", con=engine)
vaccine_schedule_df_loaded = pd.read_sql("SELECT * FROM vaccine_schedule", con=engine)

# Save DataFrames to CSV for later use
reported_cases_df.to_csv('reported_cases_data_loaded.csv', index=False)
coverage_df.to_csv('coverage_data_loaded.csv', index=False)
incidence_rate_df.to_csv('incidence_rate_data_loaded.csv', index=False)
vaccine_introduction_df_loaded.to_csv('vaccine_introduction_loaded.csv', index=False)
vaccine_schedule_df_loaded.to_csv('vaccine_schedule_loaded.csv', index=False)

Loading data from SQL Server into DataFrames...


In [7]:

reported_cases_df = pd.read_csv('reported_cases_data_loaded.csv')
coverage_df = pd.read_csv('coverage_data_loaded.csv')
incidence_rate_df = pd.read_csv('incidence_rate_data_loaded.csv')
vaccine_introduction_df_loaded = pd.read_csv('vaccine_introduction_loaded.csv')
vaccine_schedule_df_loaded = pd.read_csv('vaccine_schedule_loaded.csv')


print(f"reported_cases_df shape: {reported_cases_df.shape}")
print("reported_cases_df head:")
print(reported_cases_df.head())

print(f"coverage_df shape: {coverage_df.shape}")
print("coverage_df head:")
print(coverage_df.head())

print(f"incidence_rate_df shape: {incidence_rate_df.shape}")
print("incidence_rate_df head:")
print(incidence_rate_df.head())

print(f"vaccine_introduction_df_loaded shape: {vaccine_introduction_df_loaded.shape}")
print("vaccine_introduction_df_loaded head:")
print(vaccine_introduction_df_loaded.head())

print(f"vaccine_schedule_df_loaded shape: {vaccine_schedule_df_loaded.shape}")
print("vaccine_schedule_df_loaded head:")
print(vaccine_schedule_df_loaded.head())



reported_cases_df shape: (84870, 7)
reported_cases_df head:
      Groupn Code         Name    Year    Disease Disease_description  Cases
0  COUNTRIES  NZL  New Zealand  1983.0    MEASLES             Measles    NaN
1  COUNTRIES  NZL  New Zealand  1983.0   NTETANUS    Neonatal tetanus    NaN
2  COUNTRIES  NZL  New Zealand  1983.0  PERTUSSIS           Pertussis    0.0
3  COUNTRIES  NZL  New Zealand  1983.0      POLIO       Poliomyelitis    0.0
4  COUNTRIES  NZL  New Zealand  1983.0   TTETANUS       Total tetanus    5.0
coverage_df shape: (799718, 11)
coverage_df head:
      Groupn Code   Name    Year  Antigen  \
0  COUNTRIES  ABW  Aruba  2023.0      BCG   
1  COUNTRIES  ABW  Aruba  2023.0      BCG   
2  COUNTRIES  ABW  Aruba  2023.0  DIPHCV4   
3  COUNTRIES  ABW  Aruba  2023.0  DIPHCV4   
4  COUNTRIES  ABW  Aruba  2023.0  DIPHCV5   

                                 Antigen_description Coverage_category  \
0                                                BCG             ADMIN   
1        

In [8]:

## Handle Missing Data
print('handling missing values')
# Drop rows missing key identifiers
coverage_df = coverage_df.dropna(subset=["Year", "Code", "Name"])
reported_cases_df = reported_cases_df.dropna(subset=["Year", "Code", "Disease"])
incidence_rate_df = incidence_rate_df.dropna(subset=["Year", "Code", "Disease"])
print("done handling")

# Fill numeric columns
coverage_df["Coverage"] = coverage_df["Coverage"].fillna(0)
reported_cases_df["Cases"] = reported_cases_df["Cases"].fillna(0)
incidence_rate_df["Incidence_rate"] = incidence_rate_df["Incidence_rate"].fillna(0)

print("done filling numeric values ")


handling missing values
done handling
done filling numeric values 


In [9]:

# 🔹 2. Normalize Units

# Ensure coverage is always percentage (0–100)
coverage_df["Coverage"] = coverage_df["Coverage"].apply(
    lambda x: x*100 if 0 < x < 1 else x
)

# Standardize incidence rate per 100,000
incidence_rate_df["Incidence_rate"] = incidence_rate_df["Incidence_rate"].apply(
    lambda x: x*100000 if x < 1 else x
)
print("completed the normalization")

## Date Consistency

for df in [coverage_df, reported_cases_df, incidence_rate_df, vaccine_schedule_df_loaded, vaccine_introduction_df_loaded]:
    if "Year" in df.columns:
        df["Year"] = pd.to_numeric(df["Year"], errors="coerce").fillna(0).astype(int)
        df = df[df["Year"].between(1900, 2100)]

print("completed the data consistency")

# Example Summary

print("✅ Coverage cleaned:", coverage_df.shape)
print(coverage_df.describe())

print("✅ Reported Cases cleaned:", reported_cases_df.shape)
print(reported_cases_df.describe())

print("✅ Incidence cleaned:", incidence_rate_df.shape)
print(incidence_rate_df.describe())

# Close connections
cursor.close()
conn.close()


completed the normalization
completed the data consistency
✅ Coverage cleaned: (797168, 11)
                Year  Target_number         Doses       Coverage
count  797168.000000   1.555120e+05  1.561900e+05  797168.000000
mean     2009.214299   2.825662e+08  3.517974e+06      45.003202
std        11.719146   5.459409e+10  1.133784e+07      68.773871
min      1980.000000   0.000000e+00 -2.222882e+08       0.000000
25%      2002.000000   3.167900e+04  1.465550e+04       0.000000
50%      2012.000000   3.023390e+05  1.498490e+05      46.000000
75%      2019.000000   2.687620e+06  1.001824e+06      91.000000
max      2023.000000   1.170000e+13  1.266052e+08   32000.000000
✅ Reported Cases cleaned: (84869, 7)
               Year         Cases
count  84869.000000  8.486900e+04
mean    2004.108261  3.450124e+03
std       12.591396  5.373646e+04
min     1980.000000  0.000000e+00
25%     1994.000000  0.000000e+00
50%     2005.000000  0.000000e+00
75%     2015.000000  2.000000e+01
max     2023.0